## Check python version and tensorflow are all good

In [2]:
!python -V
!pyenv versions

Python 3.6.8
/bin/sh: 1: pyenv: not found


In [2]:
# !apt-get install wget
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-keyring_1.0-1_all.deb
# !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys F60F4B3D7FA2AF80
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A4B469963BF863CC

# Use command on terminal
# !add-apt-repository -y ppa:jblgf0/python
# !add-apt-repository --remove ppa:jblgf0/python

Executing: /tmp/tmp.wo6WkVMRQV/gpg.1.sh --keyserver
keyserver.ubuntu.com
--recv-keys
A4B469963BF863CC
gpg: requesting key 3BF863CC from hkp server keyserver.ubuntu.com
gpg: key 3BF863CC: public key "cudatools <cudatools@nvidia.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)


In [3]:
!apt update -y
!apt install build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev libreadline-dev libffi-dev wget -y
!cd /root
!wget https://www.python.org/ftp/python/3.7.16/Python-3.7.16.tgz
# from IPython.display import clear_output 
# clear_output()
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 2
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.5 1
# !sudo update-alternatives --config python3

Get:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Ign:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]0m  
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease [1581 B]
Get:5 http://security.ubuntu.com/ubuntu xenial-security InRelease [99.8 kB]m   
Hit:6 http://archive.ubuntu.com/ubuntu xenial InRelease 
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [99.8 kB]      
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Packages [518 kB]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release [564 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/r

7Progress: [ 18%] [###########...............................................] 87Progress: [ 19%] [############..............................................] 8Selecting previously unselected package libtinfo-dev:amd64.
Preparing to unpack .../libtinfo-dev_6.0+20160213-1ubuntu1_amd64.deb ...
7Progress: [ 20%] [#############.............................................] 8Unpacking libtinfo-dev:amd64 (6.0+20160213-1ubuntu1) ...
7Progress: [ 22%] [#############.............................................] 87Progress: [ 23%] [##############............................................] 8Selecting previously unselected package libncurses5-dev:amd64.
Preparing to unpack .../libncurses5-dev_6.0+20160213-1ubuntu1_amd64.deb ...
7Progress: [ 24%] [###############...........................................] 8Unpacking libncurses5-dev:amd64 (6.0+20160213-1ubuntu1) ...
7Progress: [ 25%] [###############...........................................] 87Progress: [ 26%] [################.

7Progress: [ 76%] [#############################################.............] 87Progress: [ 77%] [##############################################............] 8Setting up libnspr4-dev (2:4.13.1-0ubuntu0.16.04.1) ...
7Progress: [ 79%] [##############################################............] 87Progress: [ 80%] [###############################################...........] 8Setting up libreadline6-dev:amd64 (6.3-8ubuntu2) ...
7Progress: [ 81%] [################################################..........] 87Progress: [ 82%] [################################################..........] 8Setting up libreadline-dev:amd64 (6.3-8ubuntu2) ...
7Progress: [ 83%] [#################################################.........] 87Progress: [ 84%] [##################################################........] 8Setting up libssl-dev:amd64 (1.0.2g-1ubuntu4.20) ...
7Progress: [ 86%] [##################################################........] 87Progress: [ 87%] [############################

In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

1.15.0
/device:GPU:0


In [3]:
import pandas as pd
import numpy as np
import pysbd # Sentence segmenter
import time, random, os, sys

In [5]:
# Import mongodb_forRead.py & PredictModel.py & secret.py
from modules.mongodb_forRead import MongoDB

# Database connection
mongoDB_connection = MongoDB()
currTime = int(time.time())
today = time.localtime(currTime)
today_to_timestamp = time.mktime((today.tm_year, today.tm_mon, today.tm_mday,0,0,0,0,0,0)) # mktime -> GTC+8 timestamp
prevDayNews = mongoDB_connection.read_prev24hrs_news(today_to_timestamp)
if not prevDayNews: 
    print("There is no esg-related news today!")
    sys.exit()

# ESG News sentence segmenter
esgNews = pd.Series([],dtype=pd.StringDtype()) 
seg = pysbd.Segmenter(language="en", clean=False)
for eachNew in prevDayNews:
    newSentences = seg.segment(eachNew)
    esgNews = pd.concat([esgNews, pd.Series([sentence.replace('\xa0',' ') for sentence in newSentences])], ignore_index=True)

# Write to TSV file
date = str(today.tm_year) + str(today.tm_mon) + str(today.tm_mday)
# esgNews.to_csv("./esgBERT_input/esgNews_{}.tsv".format(date), sep='\t', header=False) # Don't try to write csv in playground
print("esgBERT input is ready!")
esgNews

esgBERT input is ready!


0     Oceans absorb up to 3 billion metric tons  of ...
1     This is a problem because when carbon dioxide ...
2     Now, due to the rapid increase in global carbo...
3     This would spell the end for numerous marine o...
4     Indeed, a thriving ocean is essential to the l...
                            ...                        
85    Kaiser Aluminum Corporation (NASDAQ: KALU), a ...
86    The report is available on the Company’s websi...
87    “We continued to build on the core elements of...
88    “Our accomplishments are a result of our conti...
89    Harvey continued, “Looking ahead, we will rema...
Length: 90, dtype: object

In [ ]:
class PredictModel:
    def __init__(self):
        self.config_path = "./predict_folder_config.json"
        with open('./import_data/label.json') as f:
            label_dict = json.load(f)
        # self.model_list = ['Carbon Emissions', 'Board', 'Privacy and Data Security', 'Supply Chain Labor Standards']
        self.model_list = list(label_dict.values())[:-1]
        self.msci_esg = {'Environment': ['Biodiversity and Land Use', 'Carbon Emissions', 'Climate Change Vulnerability', 'Electronic Waste', 'Financing Environmental Impact', 'Opportunities in Clean Tech', 'Opportunities in Green Building', 'Opportunities in Renewable Energy', 'Packaging Material and Waste', 'Product Carbon Footprint', 'Raw Material Sourcing', 'Toxic Emissions and Waste', 'Water Stress'], 'Social': ['Access to Communications', 'Access to Finance', 'Access to Health Care', 'Chemical Safety', 'Community Relations', 'Controversial Sourcing', 'Financial Product Safety', 'Health and Demographic Risk', 'Health and Safety', 'Human Capital Development', 'Labor Management', 'Opportunities in Nutrition and Health', 'Privacy and Data Security', 'Product Safety and Quality', 'Responsible Investment', 'Supply Chain Labor Standards'], 'Governance' : ['Accounting', 'Board', 'Business Ethics', 'Ownership and Control', 'Pay', 'Tax Transparency'] }
        with open(self.config_path) as f:
            self.config_dict = json.load(f)

    def get_config(self):
        with open(self.config_path) as f:
            config_dict = json.load(f)
        return config_dict
    
    def set_config(self, config_dict):
        with open(self.config_path, 'w') as f:
            json.dump(config_dict,f)
        print("Set Config Success")
        
    # Sometime the bert script will crash for no reason, and it will work after running again
    def run_model(self,file_name):
        fail_list = []
        for model in self.model_list:
            if not path.exists("{}/{}".format(self.config_dict['output_dir'],model)):
                print("Not exist")
                os.system('mkdir \"{}/{}\"'.format(self.config_dict["output_dir"],model))
            bert_script = ["python", '{}'.format(self.config_dict["script_name"]), '--task_name=class_2', '--do_predict=True', 
                           '\"--data_dir={}/{}\"'.format(self.config_dict["data_dir"],file_name), '\"--vocab_file={}/vocab.txt\"'.format(self.config_dict["pretrain_model_dir"]),
                           '\"--bert_config_file={}/bert_config.json\"'.format(self.config_dict["pretrain_model_dir"]),
                           '\"--init_checkpoint={}/{}/{}\"'.format(self.config_dict["finetune_mode_dir"],model,self.config_dict["checkpoint"]),
                           '--max_seq_length={}'.format(self.config_dict["max_seq_length"]),
                           '\"--output_dir={}/{}\"'.format(self.config_dict["output_dir"],model),
                           '\"--output_tsv_name={}\"'.format(file_name)
                          ]
            print(' '.join(bert_script))
            if os.system(' '.join(bert_script)) != 0:
                fail_list.append(model)

        if len(fail_list) == 0:
            return 'All model predict Success'
        else:
            return "{} fail!".format('/'.join(fail_list))
    
    def combine_predict_result(self, file_name):
        report_df = pd.DataFrame()
        for key_issue in self.model_list:
            tmp_df = pd.read_csv('{}/{}/{}'.format(self.config_dict["output_dir"],key_issue,file_name),sep='\t', names=['No',key_issue])
            if report_df.empty:
                report_df = tmp_df[key_issue].to_frame(name=key_issue)
            else:
                report_df[key_issue] = tmp_df[key_issue].values

        report_df.to_csv('{}/{}_table.csv'.format(self.config_dict["output_table_dir"], file_name[:-4])) 

In [ ]:
from modules.PredictModel import PredictModel
model = PredictModel()
model.get_config()

In [ ]:
currTime = int(time.time())
today = time.localtime(currTime)
today = str(today.tm_year) + str(today.tm_mon) + str(today.tm_mday)
# predict_msg = model.run_model("esgNews_{}.tsv".format(today))
predict_msg = model.run_model("esgNews_20221220.tsv".format(today))
print(predict_msg)

In [24]:
import pandas as pd
import numpy as np
# result_1_df = pd.read_csv("./esgBERT_output/esgNews_{}_table.csv".format(today), index_col=0)
result_1_df = pd.read_csv("./esgBERT_output/esgNews_20221228_table.csv", index_col=0)
result_1_df['max'] = result_1_df.max(axis=1)
filter_df = result_1_df[result_1_df['max'] >= 0.9].iloc[:,:-1]
#沒有加columns就會是全部的index
one_hot_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns).T
#one_hot_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns, columns = filter_df.index).T
all_df = one_hot_df.sum().to_frame(name="Counts")
#count_df.index.name = "Key Issue"
# all_df = all_df.T
all_df.index.name = "Key Issues"
display(all_df)

,Counts
Key Issues,
Biodiversity and Land Use,22
Carbon Emissions,56
Climate Change Vulnerability,0
Electronic Waste,0
Financing Environmental Impact,0
Opportunities in Clean Tech,0
Opportunities in Green Building,0
Opportunities in Renewable Energy,9
Packaging Material and Waste,3


In [16]:
one_hot_df.loc[one_hot_df[all_df.idxmax().values[0]] == 1].index

Int64Index([0, 2], dtype='int64')

In [17]:
one_hot_df

Key Issues,Biodiversity and Land Use,Carbon Emissions,Climate Change Vulnerability,Electronic Waste,Financing Environmental Impact,Opportunities in Clean Tech,Opportunities in Green Building,Opportunities in Renewable Energy,Packaging Material and Waste,Product Carbon Footprint,...,Financial Product Safety,Health and Demographic Risk,Health and Safety,Human Capital Development,Labor Management,Opportunities in Nutrition and Health,Privacy and Data Security,Product Safety and Quality,Responsible Investment,Supply Chain Labor Standards
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
esgNews = pd.read_csv("./esgBERT_input/esgNews_20221229.tsv", sep='\t', names=['idx', 'sentence'], index_col=0)
esgNews.iloc[2]["sentence"]

'Your favorite climate thought leaders fled Twitter once Elon Musk took over in October. '

In [3]:
esgNews

,sentence
idx,
0,You’re refreshing your go-to news sites every ...
1,"You’re over the gloom, doom and ""both sides-is..."
2,Your favorite climate thought leaders fled Twi...
3,Where can you find unique takes and data-drive...
4,Check your email.
...,...
182,"Up to 2020, investments in CCS amounted to onl..."
183,The Intergovernmental Panel on Climate Change ...
184,Will CCS be a critical part of reducing emissi...


In [4]:
most_ky_sentences = [esgNews.iloc[idx]["sentence"] for idx in one_hot_df.loc[one_hot_df[all_df.idxmax().values[0]] == 1].index]
# most_ky_sentences = [esgNews.iloc[idx]["sentence"] for idx in one_hot_df.loc[one_hot_df["Human Capital Development"] == 1].index]
most_ky_sentences

['You’re refreshing your go-to news sites every few hours, yet the stories remain stale. ',
 'You’re over the gloom, doom and "both sides-ism" that stymies climate reporting by big-time media outlets. ',
 'GreenBiz analysts and editors breathe the business of sustainability. ',
 'She’s launching Substack chats Thursdays. ',
 "(You’ll see plenty of her roommate’s dog, Fish, too.)Highlights:How journalists uncovered power companies' ploy to buy the pressTwitter's climate vigilantesA New York Times columnist went to Greenland and discovered fossil fuel talking pointsThe curious origins of the anti-ESG movementMichael Thomas is founder of the Carbon Switch  site, which offers sound consumer advice for electrifying homes, and his writings have run in The Atlantic and Fast Company. ",
 'Highlights: I Joined 40 Clean Energy Opposition Groups on Facebook. ',
 'Britt Wray, author of the new book "Generation Dread," tackles the mental health side effects of the climate crisis head-on to help rea

In [13]:
with open("./outputfile", "w") as opfile:
    opfile.write("\n".join(most_ky_sentences))

In [5]:
import random
choose_count = 30 if len(most_ky_sentences) >= 30 else len(most_ky_sentences)
most_ky_sentences = random.sample(most_ky_sentences,choose_count) # Choose random 30 sentences from most counts key issue
most_ky_sentences

['A report by Institute for Energy Economics and Financial analysis IIEEFA finds  that the majority of 13 CCS projects currently operating have either failed entirely or captured much less CO2 than expected. ',
 'With the removal of millions of metric tons of CO2 being projected, that’s a lot of financial support. ',
 'Iberdrola will implement various initiatives at this wind farm to improve the visibility of its wind turbines and deter the presence of certain species of birds in the area, installing bird detection equipment to avoid any collisions. ',
 'The company is also projecting  profits from selling "net-zero oil" produced by injecting more CO2 into oil reservoirs. ',
 'How the fossil fuel industry seeds doomism to protect continued extractionEco-fascism is part of climate anxietyThis must-read comes from multi-hyphenate Bill McKibben, who teaches environmental journalism at Middlebury College, co-founded 350.org and recently launched Third Act to mobilize older adults on climat

In [ ]:
new_df = pd.DataFrame(np.where(filter_df.T == filter_df.T.max(), 1, 0),index=filter_df.columns).T
sentence_categories = {} # {'Key Issue': ['sentence']}
sentences_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ESG_Step4/ESGNews.tsv', header=None, sep='\t')
for column in new_df.columns:
    sentence_categories[column] = [sentences_df.loc[ind][1] for ind, value in enumerate(new_df.loc[:, column]) if value == 1]

keyIssues_df = pd.DataFrame.from_dict(sentence_categories, orient='index')
keyIssues_df.insert(0, 'count', [len(sentence_categories[column]) for column in new_df.columns])
display(keyIssues_df)

keyIssues_df.to_csv("ESGNews-sentences-sorted.csv")

In [ ]:
tmp = keyIssues_df.T
tmp = tmp[:50]
tmp.to_csv("ESGNews-50-sentences-sorted.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)
test = keyIssues_df.T
test

## GPT generation

In [6]:
import os
import openai
from modules.secret import setup_api_key

setup_api_key() # adds api key to env
openai.organization = "org-hpKR7cri3YnOlgPZqIAZXtZv"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
prompt_input = '\n'.join(most_ky_sentences) + "\nWrite about 100 words tweet to summarize the news above (IMPORTANT: under 150 characters limit):\n"
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt_input,
  temperature=0.7,
  max_tokens=200, # Suggestion max_tokens > 256
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [11]:
tweet = response['choices'][0].text
print(len(tweet), tweet)

142 
The news discusses the importance of having strong relationships with people to nurture growth and success. #LifeLessons #RelationshipsMatter


In [68]:
len(tweet)

264

## Twitter Test

In [16]:
import numpy as np
import tweepy

#variables for accessing twitter API
consumer_key='VVRuSF8zMLk7ERaCSc4ulZ4iM'
consumer_secret='KlYaRsIRH1HxBUjNq24sUts09w08dtu3QT6iioR2sWrat441PE'
access_token = "1602909982223400960-lFst1DZAYe9DWTL5NLxqCcj3oaWSXY"
access_token_secret = "gDhK48DtsCRY6xAocdHQp77cjWjcyXthRZOPZhYHph40W"

# OAuth 2.0 Bearer Token (App-Only)
# bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMCxkQEAAAAA4cdJtTzMqDXuEul4uLV3NpFiQZs%3D3sI96h49nRftDbpYg43Hrg0mEj4F4RDQQrj4NAPNaq8L0MpE3b'

# Oauth2.0 Client ID & Sercet
# ClientID = 'bGI2ZE94SGpSNE9sOHdlY19IN0o6MTpjaQ'
# Client_sercet = '1MW-luKC73asmNkWkf1AxxS8zTZsO9PKwmmfiVzlD-e2yixGY8'

In [69]:
client = tweepy.Client(
    consumer_key=consumer_key, consumer_secret=consumer_secret,
    access_token=access_token, access_token_secret=access_token_secret
)

# Replace the text with whatever you want to Tweet about
response = client.create_tweet(text=tweet[:280])

print(response)

Response(data={'id': '1605839441594970112', 'text': 'CNN\'s Jeff Licht is making changes, from killing the network\'s streaming service to focusing on https://t.co/E0WDZZvBsR. He\'s also bringing Republicans back to the network, while pushing for a "rational conversation about polarizing issues." #CNN #JeffLicht #RationalConversation'}, includes={}, errors=[], meta={})
